### Overview

We have many high-quality global climate datasets that has historic data on various climatic variables. Using cloud-hosted datasets and XArray, we can compute pixel-wise long-term trends. This analysis helps us identify hotspots experienceing extreme climate change.

We will be using the [TerraClimate](https://www.climatologylab.org/terraclimate.html) gridded dataset of monthly climate and climatic water balance at high spatial resolution (~4km grid size) with a long time-series. (1958-current). This is a large dataset that is hosted on a [THREDDS Data Server (TDS)](https://www.unidata.ucar.edu/software/tds/) and served using the [OPeNDAP](https://www.opendap.org/) (Open Data Access Protocol) protocol. XArray has built-in support to efficiently read and process OPeNDAP data where we can stream and process only the required pixels without downloading entire dataset.

This notebook shows how to extract a time-series of monthly maximum temperatures and compute per-pixel linear trend. The results are processed in a distributed matter using Dask and the results are saved as a GeoTIFF file.


### Setup and Data Download



In [ ]:
%%capture
if 'google.colab' in str(get_ipython()):
    !pip install geopandas rioxarray cartopy dask[distributed] netCDF4

In [ ]:
import cartopy
import cartopy.crs as ccrs
import os
import geopandas as gpd
import xarray as xr
import rioxarray as rxr
import matplotlib.pyplot as plt
import dask

In [ ]:
data_folder = 'data'
output_folder = 'output'

if not os.path.exists(data_folder):
    os.mkdir(data_folder)
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

### Selecting a Region of Interest

[GeoBondaries](https://www.geoboundaries.org/) is an open databse of political administrative boundaries. We can download and filter for the chosen admin region.

In [ ]:
def download(url):
    filename = os.path.join(data_folder, os.path.basename(url))
    if not os.path.exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print(f'Downloaded {filename}')
    else:
        print(f'File {filename} exists.')

admin_boundaries = 'geoBoundariesCGAZ_ADM0.gpkg'

geoboundaries_url = 'https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/'
download(geoboundaries_url + admin_boundaries)


File data/geoBoundariesCGAZ_ADM0.gpkg exists.


In [ ]:
admin_boundaries_filepath = os.path.join(data_folder, admin_boundaries)
admin_gdf = gpd.read_file(admin_boundaries_filepath, encoding='utf-8')

In [ ]:
countries = ['GHA']
baltics_gdf = admin_gdf[admin_gdf.shapeGroup.isin(countries)]
baltics_gdf

In [ ]:
fig, ax = plt.subplots(1, 1)
fig.set_size_inches(5,5)
baltics_gdf.plot(
    ax=ax,
    edgecolor='#969696',
    facecolor='none',
    alpha=0.5)
ax.set_axis_off()
plt.show()

Get the bounding box to filter the XArray dataset.

In [ ]:
bounds = baltics_gdf.total_bounds
lon_min, lat_min, lon_max, lat_max = bounds

### Local Compute Cluster

Setup a Local Dask Cluster

In [ ]:
from dask.distributed import Client, progress
client = Client()  # set up local cluster on the machine
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 8.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:55407,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:55419,Total threads: 2
Dashboard: http://127.0.0.1:55423/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:55410,


2024-12-26 18:32:54,828 - distributed.nanny - WARNING - Restarting worker
2024-12-26 18:32:54,851 - distributed.nanny - WARNING - Restarting worker
2024-12-26 18:32:54,860 - distributed.nanny - WARNING - Restarting worker
2024-12-26 18:32:54,918 - distributed.nanny - WARNING - Restarting worker
2024-12-26 18:32:54,959 - distributed.nanny - WARNING - Restarting worker
2024-12-26 18:32:54,971 - distributed.nanny - WARNING - Restarting worker
2024-12-26 18:32:55,228 - distributed.nanny - WARNING - Restarting worker
2024-12-26 18:32:55,230 - distributed.nanny - WARNING - Restarting worker
2024-12-26 18:32:55,238 - distributed.nanny - WARNING - Restarting worker
2024-12-26 18:32:55,240 - distributed.nanny - WARNING - Restarting worker
2024-12-26 18:32:56,235 - distributed.nanny - WARNING - Restarting worker
2024-12-26 18:32:56,236 - distributed.nanny - WARNING - Restarting worker
2024-12-26 18:32:56,243 - distributed.nanny - WARNING - Restarting worker
2024-12-26 18:32:56,251 - distributed.

## Analysis

In [ ]:
# Choose a location within the region for plots
location = (59.4362, 24.7234) # Tallinn

### Maximum Temperature

Analyse the trend of monthly maximum temperature using TerraClimate dataset.

In [ ]:
terraclimate_url = 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/'
variable = 'tmax'
filename = f'agg_terraclimate_{variable}_1958_CurrentYear_GLOBE.nc'

remote_file_path = os.path.join(terraclimate_url, filename)
ds = xr.open_dataset(
    remote_file_path,
    chunks='auto',
    engine='netcdf4',
)
ds

<xarray.Dataset> Size: 236GB
Dimensions:  (crs: 1, lon: 8640, lat: 4320, time: 792)
Coordinates:
  * crs      (crs) int16 2B 3
  * lon      (lon) float64 69kB -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
  * lat      (lat) float64 35kB 89.98 89.94 89.9 89.85 ... -89.9 -89.94 -89.98
  * time     (time) datetime64[ns] 6kB 1958-01-01 1958-02-01 ... 2023-12-01
Data variables:
    tmax     (time, lat, lon) float64 236GB dask.array<chunksize=(65, 358, 718), meta=np.ndarray>
Attributes: (12/49)
    method:                          These layers from TerraClimate were crea...
    title:                           TerraClimate: monthly climate and climat...
    summary:                         This archive contains a dataset of high-...
    keywords:                        WORLDCLIM,global,monthly, temperature,pr...
    id:                              Blank
    naming_authority:                edu.uidaho.nkn
    ...                              ...
    geospatial_vertical_resolution:  0.0
    geospatial_vertical_positive:    Up
    references:                      Abatzoglou, J.T., S.Z. Dobrowski, S.A. P...
    source:                          WorldClim v2.0 (2.5m), CRU Ts4.0, JRA-55
    version:                         v1.0
    Conventions:                     CF-1.6

In [ ]:
# Select the variable
da = ds.tmax

# Select data within the bounding box

# Make sure the data is sorted for slice() to work correctly
da = da.sortby([da.lon, da.lat])

# Add 1 degree buffer
da_subset = da.sel(
    lon=slice(lon_min - 1, lon_max + 1),
    lat=slice(lat_min - 1, lat_max + 1)
)
da_subset

<xarray.DataArray 'tmax' (time: 792, lat: 186, lon: 223)> Size: 263MB
dask.array<getitem, shape=(792, 186, 223), dtype=float64, chunksize=(65, 186, 223), chunktype=numpy.ndarray>
Coordinates:
  * lon      (lon) float64 2kB 19.98 20.02 20.06 20.1 ... 29.1 29.15 29.19 29.23
  * lat      (lat) float64 1kB 52.94 52.98 53.02 53.06 ... 60.56 60.6 60.65
  * time     (time) datetime64[ns] 6kB 1958-01-01 1958-02-01 ... 2023-12-01
Attributes:
    units:              degC
    description:        Maximum 2-m Temperature
    long_name:          air_temperature
    standard_name:      air_temperature
    dimensions:         lon lat time
    grid_mapping:       crs
    coordinate_system:  WGS84,EPSG:4326
    _ChunkSizes:        [   1  720 1440]

Aggregate the data to yearly means.

In [ ]:
da_yearly = da_subset.groupby('time.year').mean('time')

In [ ]:
# Process and load the data into memory
# This may take a few minutes
# Check the Dask dashboard to see the progress
# The dashboard is not available on Colab
da_yearly = da_yearly.compute()

Plot a time-series at a single location to see the trend.

In [ ]:
time_series = da_yearly.interp(lat=location[0], lon=location[1])

fig, ax = plt.subplots(1, 1)

fig.set_size_inches(15, 7)
time_series.plot.line(ax=ax, x='year', marker='o', linestyle='-', linewidth=1)
ax.set_title('Annual Mean Monthly Maximum Temperature')
plt.show()

Fit a linear trendline.

In [ ]:
trend = da_yearly.polyfit('year', 1) # fit polynomial of degree 1
slope = trend.polyfit_coefficients[0,...] * 100 # per year -> per century

Visualize the slope of trendline.

In [ ]:
projection = ccrs.epsg(25884) # ETRS89 / TM Baltic93

cbar_kwargs = {
    'orientation':'horizontal',
    'fraction': 0.025,
    'pad': 0.05,
    'extend':'neither',
    'label': '°C per century'
}

fig, ax = plt.subplots(1, 1, subplot_kw={'projection': projection})
fig.set_size_inches(8, 8)
slope.plot.imshow(
    ax=ax,
    cmap='YlOrBr',
    transform=ccrs.PlateCarree(),
    add_labels=False,
    cbar_kwargs=cbar_kwargs)

ax.coastlines()
ax.set_extent((lon_min,lon_max,lat_min,lat_max), crs = ccrs.PlateCarree())

plt.title(f'Slope of Monthly Maximum Temperature Trend', fontsize = 12)
plt.show()

Save the resulting slope raster as a GeoTIFF.

In [ ]:
# Assign a CRS to the DataArray
slope.rio.write_crs('EPSG:4326', inplace=True)
# Clip to the GeoDataFrame coundary
clipped = slope.rio.clip(baltics_gdf.geometry.values)
# Write the file
output_slope_file = f'{variable}_slope.tif'
output_slope_path = os.path.join(output_folder, output_slope_file)
if not os.path.exists(output_slope_path):
    slope.rio.to_raster(output_slope_path)
    print('Saved the file at ', output_slope_path)

Save the yearly aggregated subset as a NetCDF.

In [ ]:
local_subset_file = f'{variable}_yearly_subset.nc'
local_subset_filepath = os.path.join(output_folder, local_subset_file)
if not os.path.exists(local_subset_filepath):
    da_yearly.to_netcdf(path=local_subset_filepath)
    print('Saved the file at ', local_subset_filepath)